## Navigating an environment in Unity ML-Agents using Deep Q-Networks

For this project, I use a double DQN Algorithm to train the agent as shown in [rl_algorithm](http://localhost:8888/notebooks/Deep_Reinforcement_Learning/Navigation_project/rl_algorithm.ipynb)

In [1]:
#Import libraries

from collections import deque
from unityagents import UnityEnvironment
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch

from rl_algorithm import Agent as DDQN_Agent

Initializing the environment

In [2]:
# change path if necessary
env = UnityEnvironment(file_name="./Banana_Linux/Banana.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space
state = env_info.vector_observations[0]
state_size = len(state)
print('States have length:', state_size)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of actions: 4
States have length: 37


In [3]:
# Training the agent

def train_agent(agent, n_episodes=2000, max_t=2000):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        agent.adaptive_eps.update()
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), './checkpoints/checkpoint_' + str(i_episode-100) +  '.pth')
            break
    return scores

agent = DDQN_Agent(state_size=state_size, action_size=action_size)
scores = train_agent(agent)

Episode 100	Average Score: 0.07
Episode 200	Average Score: 0.76
Episode 300	Average Score: 1.04
Episode 400	Average Score: 2.84
Episode 500	Average Score: 4.01
Episode 600	Average Score: 6.24
Episode 700	Average Score: 7.64
Episode 800	Average Score: 8.77
Episode 900	Average Score: 10.32
Episode 1000	Average Score: 12.29
Episode 1044	Average Score: 13.00
Environment solved in 944 episodes!	Average Score: 13.00


FileNotFoundError: [Errno 2] No such file or directory: './checkpoints/checkpoint_944.pth'